In [1]:
import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import numpy as np

# loading the data

In [9]:
composed = transforms.Compose([transforms.Resize((16, 16)), transforms.ToTensor()])

In [10]:
train_dataset = dsets.MNIST(root='./data', train=True, download=True, transform=composed)
validation_dataset = dsets.MNIST(root='./data', train=False, download=True, transform=composed)

In [14]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, batch_size=5000, shuffle=False)

In [15]:
image, label = train_dataset[0]
img_shape = image.shape
flattened_image = image.view(-1)
print(len(image))
print(len(flattened_image))

1
256


# CNN with 2 convolution layer

In [17]:
class CNN(nn.Module):
    
    def __init__(self, out_1=16, out_2=32):
        super(CNN, self).__init__()
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=out_1, kernel_size=5, padding=2)
        self.maxpool1=nn.MaxPool2d(kernel_size=2)

        self.cnn2 = nn.Conv2d(in_channels=out_1, out_channels=out_2, kernel_size=5, stride=1, padding=2)
        self.maxpool2=nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(out_2 * 4 * 4, 10)
    
    def forward(self, x):
        x = self.cnn1(x)
        x = torch.relu(x)
        x = self.maxpool1(x)
        x = self.cnn2(x)
        x = torch.relu(x)
        x = self.maxpool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x
    
    def train(self, model, train_data, criterion, optimizer, epochs):
        for epoch in range(epochs):
            i = 0
            for i, (x, y) in enumerate(train_loader): 
                optimizer.zero_grad()
                z = model(x)
                loss = criterion(z, y)
                loss.backward()
                optimizer.step()

    def val_acc(self, model, val_data):
        correct = 0
        for x, y in val_data:
            z = model(x)
            _, label = torch.max(z, 1)
            correct += (label == y).sum().item()
        return 100 * (correct / len(validation_dataset))

In [18]:
model = CNN(out_1=16, out_2=32)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
model.train(model, train_loader, criterion, optimizer, epochs=10)

In [19]:
print(model.val_acc(model, validation_loader))

98.64
